# go-tflite

Go binding for TensorFlow Lite

This Go package is Go binding that wrap TensorFlow Lite C library. Current status is experimental but already works with some examples. This tutorial make the inference of FizzBuzz. The model of FizzBuzz is generated using numpy/keras on Python.

```python
import numpy as np
from tensorflow.contrib.keras.api.keras.models import Sequential, model_from_json
from tensorflow.contrib.keras.api.keras.layers import Dense, Dropout, Activation
from tensorflow.contrib.keras.api.keras.optimizers import SGD, Adam
import tensorflow.contrib.lite as lite


def fizzbuzz(i):
    if   i % 15 == 0: return np.array([0, 0, 0, 1])
    elif i % 5  == 0: return np.array([0, 0, 1, 0])
    elif i % 3  == 0: return np.array([0, 1, 0, 0])
    else:             return np.array([1, 0, 0, 0])

def bin(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

NUM_DIGITS = 7
trX = np.array([bin(i, NUM_DIGITS) for i in range(1, 101)])
trY = np.array([fizzbuzz(i) for i in range(1, 101)])
model = Sequential()
model.add(Dense(64, input_dim = 7))
model.add(Activation('tanh'))
model.add(Dense(4, input_dim = 64))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(trX, trY, epochs = 3600, batch_size = 64)
model.save('fizzbuzz_model.h5')

converter = lite.TFLiteConverter.from_keras_model_file('fizzbuzz_model.h5')
tflite_model = converter.convert()
open('fizzbuzz_model.tflite', 'wb').write(tflite_model)
```

At the first, import packages.

In [1]:
import (
    "fmt"
    "log"
)

Add utility functions to encode/decode numbers since the number must be 1 hot.

In [2]:
func bin(n int, num_digits int) []float32 {
    f := make([]float32, num_digits)
    for i := 0; i < num_digits; i++ {
        f[i] = float32((n >> uint(i)) & 1)
    }
    return f[:]
}

func dec(b []float32) int {
    for i := 0; i < len(b); i++ {
        if b[i] > 0.4 {
            return i
        }
    }
    panic("Sorry, I'm wrong")
}

Next, import go-tflite. If you use [gophernotes](https://github.com/gopherdata/gophernotes) kernel on Jupyter Notebook, you must generate go-tflite as third_party. See the https://github.com/cosmos72/gomacro#importing-packages  

In [3]:
import "github.com/mattn/go-tflite"

Load tflite model file that is generated by Python.

In [4]:
model := tflite.NewModelFromFile("fizzbuzz_model.tflite")
if model == nil {
    log.Fatal("cannot load model")
}

 Now you can create interpreter of TensorFlow Lite. The second argument is InterpreterOptions. If you want to specify thread number of the interpreter, modify options that created by NewInterpreterOptions.
 ```go
options := tflite.NewInterpreterOptions()
options.SetNumThread(4)
defer options.Delete()

interpreter := tflite.NewInterpreter(model, options)
```

In [5]:
interpreter := tflite.NewInterpreter(model, nil)

Allocate tensors according to the model.

In [6]:
if status := interpreter.AllocateTensors(); status != tflite.OK {
    log.Fatal("cannot allocate tensors")
}

Now interpreter have input/output tensors. You can update input tensor.

In [7]:
input := interpreter.GetInputTensor(0)

So the `input` is a slice of values that can make sure the type with `Type()`.

In [8]:
input.Type()

You can update this slice directly. For example, update with 5 (Buzz) using `bin` defined above.

In [9]:
bin(5, 7)

In [10]:
copy(input.Float32s(), bin(5, 7))

The, call `Invoke()`

In [11]:
if status := interpreter.Invoke(); status != tflite.OK {
    log.Fatal("cannot invoke")
}

Now you can access output tensor.

In [12]:
output := interpreter.GetOutputTensor(0)

In [13]:
output.Float32s()

[0.035052534 0.03628092 0.92866534 1.2214607e-06]

The numbers can be decoded with `dec` above.

In [14]:
dec(output.Float32s())

2

This `2` mean `Buzz` that is encoded for the number that can be divided by 5.

```python
def fizzbuzz(i):
    if   i % 15 == 0: return np.array([0, 0, 0, 1])
    elif i % 5  == 0: return np.array([0, 0, 1, 0])
    elif i % 3  == 0: return np.array([0, 1, 0, 0])
    else:             return np.array([1, 0, 0, 0])
```

This is an offset to the of four values that makes pulse 1. 
The binary encodeding makes reversed values:

* `0001` is `1` that indicate `Fizz`
* `0010` is `2` that indicate `Buzz`
* `0100` is `3` that indicate `FizzBuzz`
* `1000` is `4` that indicate original number. i.e. `5`.

Let's add function to display Fizz, Buzz, FizzBuzz or input number.

In [15]:
func display(v []float32, i int) {
    switch dec(v) {
    case 0:
        fmt.Println(i)
    case 1:
        fmt.Println("Fizz")
    case 2:
        fmt.Println("Buzz")
    case 3:
        fmt.Println("FizzBuzz")
    }
}

In [16]:
display(output.Float32s(), 5)

Buzz


Finally, let's display FizzBuzz with one hundred numbers.

In [17]:
for i := 1; i <= 100; i++ {
    copy(interpreter.GetInputTensor(0).Float32s(), bin(i, 7))
    interpreter.Invoke()
    display(interpreter.GetOutputTensor(0).Float32s(), i)
}

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz
16
17
Fizz
19
Buzz
Fizz
22
23
Fizz
Buzz
26
Fizz
28
29
FizzBuzz
31
32
Fizz
34
Buzz
Fizz
37
38
Fizz
Buzz
41
Fizz
43
44
FizzBuzz
46
47
Fizz
49
Buzz
Fizz
52
53
Fizz
Buzz
56
Fizz
58
59
FizzBuzz
61
62
Fizz
64
Buzz
Fizz
67
68
Fizz
Buzz
71
Fizz
73
74
FizzBuzz
76
77
Fizz
79
Buzz
Fizz
82
83
Fizz
Buzz
86
Fizz
88
89
FizzBuzz
91
92
Fizz
94
Buzz
Fizz
97
98
Fizz
Buzz


Machine learning is fun, isn't it?